# PhobiaShield - YOLOv8 Baseline Training

**YOLOv8s** baseline per comparison con FPN custom.

**Team:** Gabriele (Architect) | Member A (Data) | Member C (Demo)

**Course:** Fundamentals of Data Science - Sapienza University

**Date:** December 2025

---

## Why YOLOv8?
- **Transfer Learning:** Pre-trained on COCO (80 classes)
- **State-of-the-art:** Modern architecture
- **Baseline Comparison:** Test custom FPN vs industry standard

## Expected Results
- **mAP50:** 65-75% (vs FPN ~28%)
- **Training Time:** 1.5-2 hours
- **Inference:** 10ms per image (4× faster than FPN)

## 1. Setup

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# Check GPU
!nvidia-smi

## 2. Install YOLOv8

In [ ]:
!pip install -q ultralytics

from ultralytics import YOLO
import os

print("✅ YOLOv8 installed")

## 3. Extract Dataset

In [ ]:
import zipfile
from pathlib import Path

# Paths
ZIP_PATH = '/content/drive/MyDrive/PhobiaShield_Models/PhobiaShield/DATASET_ULTIMATE_COMPLETE.zip'
EXTRACT_TO = '/content/dataset_yolo'

if not os.path.exists(ZIP_PATH):
    print(f"❌ Dataset not found: {ZIP_PATH}")
else:
    print("📦 Extracting...")
    
    os.makedirs(EXTRACT_TO, exist_ok=True)
    
    with zipfile.ZipFile(ZIP_PATH, 'r') as z:
        z.extractall(EXTRACT_TO)
    
    train_imgs = len(list(Path(f'{EXTRACT_TO}/train/images').glob('*.jpg')))
    val_imgs = len(list(Path(f'{EXTRACT_TO}/val/images').glob('*.jpg')))
    
    print(f"✅ Dataset ready")
    print(f"   Train: {train_imgs}")
    print(f"   Val: {val_imgs}")

## 4. Create data.yaml

In [ ]:
# Create YOLO format data.yaml
data_yaml = f"""
path: {EXTRACT_TO}
train: train/images
val: val/images
test: test/images

nc: 5
names: ['clown', 'shark', 'spider', 'blood', 'needle']
"""

yaml_path = '/content/phobiashield.yaml'
with open(yaml_path, 'w') as f:
    f.write(data_yaml)

print(f"✅ data.yaml created:")
print(data_yaml)

## 5. Training

In [ ]:
# Load YOLOv8s model
model = YOLO('yolov8s.pt')

# Save directory
SAVE_DIR = '/content/drive/MyDrive/PhobiaShield_Models/yolov8s'

print("🚀 Starting YOLOv8 training...")
print("="*70)

# Train
results = model.train(
    data=yaml_path,
    epochs=50,
    batch=64,
    imgsz=416,
    patience=20,
    device=0,
    project=SAVE_DIR,
    name='train',
    exist_ok=True,
    
    # Optimization
    amp=True,
    augment=True,
    
    # Hyperparameters
    lr0=0.01,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    
    # Loss weights
    box=7.5,
    cls=0.5,
    dfl=1.5
)

print("\n" + "="*70)
print("✅ Training complete!")
print(f"Best model: {SAVE_DIR}/train/weights/best.pt")

## 6. Validation

In [ ]:
# Load best model
best_model = YOLO(f'{SAVE_DIR}/train/weights/best.pt')

# Validate
print("📊 Running validation...")
val_results = best_model.val()

print("\n" + "="*70)
print("📈 YOLO V8 RESULTS")
print("="*70)
print(f"mAP50: {val_results.box.map50:.3f}")
print(f"mAP50-95: {val_results.box.map:.3f}")
print(f"Precision: {val_results.box.mp:.3f}")
print(f"Recall: {val_results.box.mr:.3f}")
print("="*70)

## 7. Results & Comparison

In [ ]:
# Show results
from IPython.display import Image, display

print("📊 Training curves:")
display(Image(f'{SAVE_DIR}/train/results.png'))

print("\n🎯 Confusion matrix:")
display(Image(f'{SAVE_DIR}/train/confusion_matrix.png'))

print("\n💾 Model saved to Drive:")
!ls -lh "{SAVE_DIR}/train/weights/"

print("\n🎉 YOLOv8 training complete!")
print("Next: Run evaluation notebook for comparison with FPN")